In [1]:
import pandas as pd
from mainnet_launch.accounting.incentive_token_liqudiation_prices import (
    _build_swapped_df,
    ETH_CHAIN,
    BASE_CHAIN,
    _fetch_incentive_token_price_df,
)

import pandas as pd
import streamlit as st
from multicall import Call
import plotly.express as px
import plotly.subplots as sp
import plotly.graph_objects as go
from datetime import datetime, timedelta, timezone


from mainnet_launch.data_fetching.get_state_by_block import (
    get_raw_state_by_blocks,
    get_state_by_one_block,
    identity_with_bool_success,
    safe_normalize_with_bool_success,
    build_blocks_to_use,
)
from mainnet_launch.data_fetching.add_info_to_dataframes import add_timestamp_to_df_with_block_column
from mainnet_launch.data_fetching.get_events import fetch_events

from mainnet_launch.constants import (
    INCENTIVE_PRICNIG_STATS,
    LIQUIDATION_ROW,
    eth_client,
    CACHE_TIME,
    ROOT_PRICE_ORACLE,
    AutopoolConstants,
    ChainData,
    ETH_CHAIN,
    BASE_CHAIN,
)

from mainnet_launch.abis.abis import DESTINATION_DEBT_REPORTING_SWAPPED_ABI


def _add_acheived_price_column(swapped_df: pd.DataFrame, token_address_to_decimals: dict) -> dict:
    def _compute_achieved_price(row):
        sell_token_decimals = token_address_to_decimals[row["sellTokenAddress"]]
        normalized_sell_amount = row["sellAmount"] / (10**sell_token_decimals)

        buy_token_decimals = token_address_to_decimals[row["buyTokenAddress"]]
        normalized_buy_amount = row["buyTokenAmountReceived"] / (10**buy_token_decimals)

        achieved_price = normalized_buy_amount / normalized_sell_amount
        return achieved_price

    swapped_df["achieved_price"] = swapped_df.apply(lambda row: _compute_achieved_price(row), axis=1)

    return swapped_df


def _fetch_oracle_price_df(swapped_df: pd.DataFrame, chain: ChainData) -> pd.DataFrame:
    token_addresses = [*swapped_df["sellTokenAddress"].unique(), *swapped_df["buyTokenAddress"].unique()]
    oracle_price_calls = [
        Call(
            ROOT_PRICE_ORACLE(chain),
            ["getPriceInEth(address)(uint256)", addr],
            [(addr, safe_normalize_with_bool_success)],
        )
        for addr in token_addresses
    ]

    blocks = swapped_df["block"].unique()
    oracle_price_df = get_raw_state_by_blocks(oracle_price_calls, blocks, chain, include_block_number=True)
    return oracle_price_df


def _fetch_incentive_calculator_price_df(swapped_df: pd.DataFrame, chain: ChainData) -> pd.DataFrame:
    token_addresses = [*swapped_df["sellTokenAddress"].unique(), *swapped_df["buyTokenAddress"].unique()]

    def _min_of_low_and_high_price(success, data):
        if success:
            fast, slow = data
            return min(fast, slow) / 1e18
        return None

    oracle_price_calls = [
        Call(
            INCENTIVE_PRICNIG_STATS(chain),
            ["getPrice(address,uint40)((uint256,uint256))", addr, 2 * 86400],  # 2 days of latency
            [(addr, _min_of_low_and_high_price)],
        )
        for addr in token_addresses
    ]

    blocks = swapped_df["block"].unique()
    incentive_calculator_price_df = get_raw_state_by_blocks(
        oracle_price_calls, blocks, chain, include_block_number=True
    )
    return incentive_calculator_price_df


def fetch_swapped_event_df_with_acheived_price(chain: ChainData, start_block: int) -> pd.DataFrame:
    contract = chain.client.eth.contract(LIQUIDATION_ROW(chain), abi=DESTINATION_DEBT_REPORTING_SWAPPED_ABI)

    swapped_df = fetch_events(contract.events.Swapped, start_block=start_block)
    oracle_price_df = _fetch_oracle_price_df(swapped_df, chain)
    incentive_calculator_price_df = _fetch_incentive_calculator_price_df(swapped_df, chain)

    token_addresses = [*swapped_df["sellTokenAddress"].unique(), *swapped_df["buyTokenAddress"].unique()]
    symbol_calls = [Call(addr, ["symbol()(string)"], [(addr, identity_with_bool_success)]) for addr in token_addresses]
    token_address_to_symbol = get_state_by_one_block(symbol_calls, swapped_df["block"].max(), chain)

    decimals_calls = [
        Call(addr, ["decimals()(uint8)"], [(addr, identity_with_bool_success)]) for addr in token_addresses
    ]

    token_address_to_decimals = get_state_by_one_block(decimals_calls, swapped_df["block"].max(), chain)

    swapped_df = _add_acheived_price_column(swapped_df, token_address_to_decimals)

    long_oracle_prices_df = pd.melt(
        oracle_price_df,
        id_vars=["block"],  # Keep 'block' as it is
        var_name="sellTokenAddress",  # Name for the token address column
        value_name="oracle_price",  # Name for the value column
    )
    long_swapped_df = swapped_df[["block", "sellTokenAddress", "achieved_price"]].copy()

    long_incentive_calculator_prices = pd.melt(
        incentive_calculator_price_df,
        id_vars=["block"],  # Keep 'block' as it is
        var_name="sellTokenAddress",  # Name for the token address column
        value_name="incentive_calculator_price",  # Name for the value column
    )

    long_swapped_df = swapped_df[["block", "sellTokenAddress", "achieved_price"]].copy()
    long_swapped_df = pd.merge(long_swapped_df, long_oracle_prices_df, on=["block", "sellTokenAddress"], how="left")
    long_swapped_df = pd.merge(
        long_swapped_df, long_incentive_calculator_prices, on=["block", "sellTokenAddress"], how="left"
    )

    timestamp_df = oracle_price_df.reset_index()[["timestamp", "block"]]

    long_swapped_df = pd.merge(long_swapped_df, timestamp_df, on=["block"], how="left")
    long_swapped_df["tokenSymbol"] = long_swapped_df["sellTokenAddress"].apply(lambda x: token_address_to_symbol[x])
    long_swapped_df["chain"] = chain.name

    return long_swapped_df


# step 1
# load it from disk if it exits,
# if it does,

ImportError: cannot import name '_build_swapped_df' from partially initialized module 'mainnet_launch.accounting.incentive_token_liqudiation_prices' (most likely due to a circular import) (/home/parker/Documents/Tokemak/v2-rebalance-dashboard/mainnet_launch/accounting/incentive_token_liqudiation_prices.py)

In [19]:
from mainnet_launch.data_fetching.databases import write_df_to_table, load_table_if_exists

long_swapped_df = fetch_swapped_event_df_with_acheived_price(BASE_CHAIN, start_block=21241103 - 10_000)
# multiple runs should not add more to the db
INCENTIVE_TOKEN_TABLE_NAME = "INCENTIVE_TOKEN_PRICES_AT_LIQUIDATION"
write_df_to_table(long_swapped_df, INCENTIVE_TOKEN_TABLE_NAME)

2024-12-13 14:10:18.437 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-13 14:10:18.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-13 14:10:18.438 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-13 14:10:18.439 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2024-12-13 14:10:18.440 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


Table 'INCENTIVE_TOKEN_PRICES_AT_LIQUIDATION' Does not exist so it is created and data inserted.


In [20]:
loaded_df = load_table_if_exists(INCENTIVE_TOKEN_TABLE_NAME, where_clause=None)
loaded_df

,block,sellTokenAddress,achieved_price,oracle_price,incentive_calculator_price,timestamp,tokenSymbol,chain
0,22063513,0x1509706a6c66CA549ff0cB464de88231DDBe213B,0.000109,0.000111,0.000110,2024-11-06 18:06:13+00:00,AURA,base
1,22063513,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1,0.000719,0.000725,0.000720,2024-11-06 18:06:13+00:00,BAL,base
2,22086591,0x1509706a6c66CA549ff0cB464de88231DDBe213B,0.000106,0.000106,0.000109,2024-11-07 06:55:29+00:00,AURA,base
3,22086591,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1,0.000694,0.000711,0.000722,2024-11-07 06:55:29+00:00,BAL,base
4,22118991,0x1509706a6c66CA549ff0cB464de88231DDBe213B,0.000102,0.000101,0.000105,2024-11-08 00:55:29+00:00,AURA,base
...,...,...,...,...,...,...,...,...
165,23587791,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1,0.000930,0.000971,0.000896,2024-12-12 00:55:29+00:00,BAL,base
166,23587791,0x940181a94A35A4569E4529A3CDfB74e38FD98631,0.000575,0.000576,0.000505,2024-12-12 00:55:29+00:00,AERO,base
167,23609393,0x1509706a6c66CA549ff0cB464de88231DDBe213B,0.000114,0.000115,0.000113,2024-12-12 12:55:33+00:00,AURA,base
168,23609393,0x4158734D47Fc9692176B5085E0F52ee0Da5d47F1,0.000925,0.000951,0.000900,2024-12-12 12:55:33+00:00,BAL,base


In [24]:
loaded_df.drop_duplicates().shape, loaded_df.shape

((170, 8), (170, 8))

In [22]:
def _fetch_incentive_token_price_df(swapped_df: pd.DataFrame, chain: ChainData):
    symbol_calls = [
        Call(addr, ["symbol()(string)"], [(addr, identity_with_bool_success)])
        for addr in swapped_df["sellTokenAddress"].unique()
    ]
    block = max(swapped_df["block"])
    token_address_to_symbol = get_state_by_one_block(symbol_calls, block, chain=chain)

    def _min_of_low_and_high_price(success, data):
        if success:
            fast, slow = data
            return min(fast, slow) / 1e18
        return None

    def getIncentiveTokenPrice(name: str, token_address: str) -> Call:
        return Call(
            INCENTIVE_PRICNIG_STATS(chain),
            ["getPrice(address,uint40)((uint256,uint256))", token_address, 2 * 86400],  # 2 days
            [(name, _min_of_low_and_high_price)],
        )

    def getOraclePrice(name: str, token_address: str) -> Call:
        return Call(
            ROOT_PRICE_ORACLE(chain),
            ["getPriceInEth(address)(uint256)", token_address],
            [(name, safe_normalize_with_bool_success)],
        )

    blocks_to_get_incentive_token_prices = swapped_df["block"].unique()

    incentive_stats_calls = [getIncentiveTokenPrice(symbol, addr) for addr, symbol in token_address_to_symbol.items()]
    incentive_stats_token_prices_df = get_raw_state_by_blocks(
        incentive_stats_calls, blocks_to_get_incentive_token_prices, chain, include_block_number=True
    )

    oracle_price_calls = [getOraclePrice(symbol, addr) for addr, symbol in token_address_to_symbol.items()]
    oracle_price_df = get_raw_state_by_blocks(
        oracle_price_calls, blocks_to_get_incentive_token_prices, chain, include_block_number=True
    )

    achieved_eth_price_df = pd.pivot(
        swapped_df, index="block", columns="sellTokenAddress", values="achieved_token_price_in_eth"
    )

    achieved_eth_price_df.columns = [token_address_to_symbol[c] for c in achieved_eth_price_df.columns]

    return incentive_stats_token_prices_df, oracle_price_df, achieved_eth_price_df